# Welcome to your first assignment!

Day 3 Competition of the Frontier LLMs - My Version 

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">LLM Competition Assignment</h2>
        </td>
    </tr>
</table>

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import ollama

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

In [ ]:
openai = OpenAI()

In [ ]:
def make_llm_call(model_name, messages):
    if model_name == "gemini-2.0-flash":
        gemini = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
        response = gemini.beta.chat.completions.parse(model=model_name, messages=messages)
    else:
        response = openai.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

In [ ]:
def make_pitch(group_members):
    member_pitches = []
    for member in group_members:
        system_prompt = f""" You are a member of a group and your name is {member["name"]}."""
        user_prompt = f""" You are required to perform group functions. But for now you are required to make a pich of not more than 150 words on why you should be the leader of the group"""
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        pitch = make_llm_call(model_name=member["model_name"], messages=messages)
        member_pitches.append({"name": member["name"], "pitch": pitch, "model_name": member["model_name"]})
        # display(messages)
    return member_pitches

In [ ]:
def vote_members(member_pitches):
    """
    This function allows a member to vote for the other members excluding themselves
    """

    votes = []
    for member in member_pitches:
        members_to_vote = [m for m in member_pitches if m["name"] != member["name"]]

        system_prompt = f""" You are a member of a group and your name is {member["name"]}."""
        user_prompt = f""" You are required to perform group functions. But for now you are required to vote
         for the members based on their pitch. This contains the member and the pitch {members_to_vote}.
         
         Once you reviewed the pitch and decided who to vote for, Return ONLY a JSON object exactly like this (no extra text):
         {{"selected": "<name>"}}
         """
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        vote = make_llm_call(model_name=member["model_name"], messages=messages)
        votes.append({"name": member["name"], "voted_for": vote})
    return votes

In [ ]:
def declare_winner(votes):
    """
    This function declares the winner of the competition
    """
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

    system_prompt = """ You are the electoral commission and your role is to declare a winner of the group election"""
    user_prompt = f""" You are required to declare a winner of the group election. The votes are {votes}""" 

    resp = ollama_via_openai.chat.completions.create(
    model="llama3.2:latest",
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ])


    return resp.choices[0].message.content

In [ ]:
from IPython.display import Markdown


group_members = [
    {"name": "Paul", "model_name": "gemini-2.0-flash"},
    {"name": "Yoku", "model_name": "gpt-4o-mini"},
    {"name": "Sheila", "model_name": "gpt-4o-mini"}
]

member_pitches = make_pitch(group_members)

print(member_pitches)

votes = vote_members(member_pitches)

print(votes)

winner = declare_winner(votes)

display(Markdown(winner))